In [1]:
!pip install -q ultralytics onnx onnxruntime onnxruntime-gpu

In [2]:
import os
import sys
from pathlib import Path

BASE_DIR = Path.cwd().parents[0]   # notebooks -> PCB
WEIGHTS_DIR = BASE_DIR / "weights"
os.chdir(BASE_DIR)

print("BASE_DIR:", BASE_DIR)
print("WEIGHTS_DIR:", WEIGHTS_DIR)
print(f"Current working directory: {os.getcwd()}")
print("files:", list(WEIGHTS_DIR.glob("*.pt")))

BASE_DIR: /content/drive/MyDrive/likelion/PCB
WEIGHTS_DIR: /content/drive/MyDrive/likelion/PCB/weights
Current working directory: /content/drive/MyDrive/likelion/PCB
files: [PosixPath('/content/drive/MyDrive/likelion/PCB/weights/best_yolov11m_imgsz1280.pt')]


In [4]:
from src.inference.quantization import Quantizer, quantize_model

WEIGHTS_PATH = WEIGHTS_DIR / "best_yolov11m_imgsz1280.pt"
quantizer = Quantizer(weights_path=str(WEIGHTS_PATH))
quantizer.export_fp16()

print("Quantized files:")
for f in WEIGHTS_DIR.glob("*"):
    print(" -", f.name)

Ultralytics 8.4.2 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11m summary (fused): 125 layers, 20,034,658 parameters, 0 gradients, 67.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/likelion/PCB/weights/best_yolov11m_imgsz1280.pt' with input shape (1, 3, 1280, 1280) BCHW and output shape(s) (1, 10, 33600) (38.8 MB)
requirements: Ultralytics requirement ['onnxslim>=0.1.71'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 10 packages in 291ms
Prepared 2 packages in 45ms
Installed 2 packages in 4ms
 + colorama==0.4.6
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 0.5s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.20.1 opset 20...
ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 3.4s, 

### Inference

In [ ]:
TEST_DIR = BASE_DIR / "dataset/roboflow/test/images"
MODEL_PT = WEIGHTS_DIR / "best_yolov11m_imgsz1280.pt"
MODEL_ENGINE = WEIGHTS_DIR / "best_yolov11m_imgsz1280.engine"

test_images = list(TEST_DIR.glob("*.jpg")) + list(TEST_DIR.glob("*.png"))
print(f"Found {len(test_images)} test images")

In [ ]:
import time
import torch
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO

def load_image(img_path):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def evaluate_model(model, images, conf=0.25, imgsz=1280):
    inference_times = []
    ng_count = 0

    for img_path in images:
        img = load_image(img_path)

        t0 = time.time()
        results = model.predict(img, imgsz=imgsz, conf=conf, verbose=False)[0]
        t1 = time.time()
        inference_times.append(t1 - t0)

        # NG 판정: box가 하나라도 있으면 NG
        if len(results.boxes) > 0:
            ng_count += 1

    avg_time = np.mean(inference_times)
    fps = 1.0 / avg_time if avg_time > 0 else 0
    total_images = len(images)
    ok_count = total_images - ng_count

    return {
        "avg_inference_time": avg_time,
        "FPS": fps,
        "total_images": total_images,
        "NG": ng_count,
        "OK": ok_count,
        "NG_rate": ng_count / total_images * 100
    }

In [ ]:
# 원본 모델 평가
print("Evaluating original model (.pt)...")
model_pt = YOLO(str(MODEL_PT))
pt_stats = evaluate_model(model_pt, test_images)
print("Original Model Results:", pt_stats)

# TensorRT FP16 모델 평가
print("\nEvaluating TensorRT FP16 model (.engine)...")
model_trt = YOLO(str(MODEL_ENGINE))  # ultralytics engine 자동 인식
trt_stats = evaluate_model(model_trt, test_images)
print("TensorRT FP16 Model Results:", trt_stats)

# 비교표 출력
df_compare = pd.DataFrame([pt_stats, trt_stats], index=["Original PT", "TensorRT FP16"])
print("\n===== Performance Comparison =====")
df_compare